In [16]:
import json
from urllib.parse import urlparse, parse_qs, urljoin
import requests as rq
from bs4 import BeautifulSoup
import re 
from selenium import webdriver

In [17]:
def get_daylywebtoons():
    webtoon_main_url = WC['TOP_URL']
    res = rq.get(webtoon_main_url)
    main_soup = BeautifulSoup(res.content, 'lxml')
    webtoon_links = [{"title": a_tag.get('title'), "link": urljoin(WC['TOP_URL'], a_tag.get('href'))}
                      for a_tag in main_soup.select('#content a.title')]
    
    return webtoon_links


In [18]:
WC = {'TOP_URL' : 'http://comic.naver.com/webtoon/weekday.nhn'}
webtoons = get_daylywebtoons()
webtoons[20]
    

{'link': 'http://comic.naver.com/webtoon/list.nhn?titleId=687915&weekday=mon',
 'title': '꿈의 기업'}

In [15]:
def get_wt_info(genre):
    driver =  webdriver.PhantomJS()
    driver.get("http://comic.naver.com/webtoon/genre.nhn?view=list&order=ViewCount&genre=" + genre)
    wts = driver.find_elements_by_css_selector('#content tbody tr')
    
    ls =[]
    for wt in wts:
        title = wt.find_element_by_css_selector('td').text
        star_score = wt.find_element_by_css_selector('.rating_type strong').text
        mk = wt.find_element_by_css_selector('td:nth-child(3) a').text
        if True if re.search('(완결)', title) else False:
            pass
        else:
            ls.append([title, star_score, mk, genre])
   
    driver.close()
    return pd.DataFrame(data=ls,columns=['title', 'score', 'artist', 'genre'])

In [39]:
def drop_user(wt_num):
    dff = pd.read_csv('data/webtoon_{}.csv'.format(wt_num), index_col = 0)
    count_user = dff.groupby(['user_id']).size().reset_index(name='Counts').sort_values(by='Counts',ascending=False)
    print(wt_num,', number of drop_user : ',len(count_user[count_user['Counts'] < 5]))
    drop_user = list(count_user[count_user['Counts'] < 5]['user_id'])
    df = dff.copy()
    for idx in range(len(df)): 
        if df['user_id'][idx] in drop_user:
            df = df.drop(idx)
    df.to_csv('data/dropu_webtoon_{}.csv'.format(wt_num))        
    return df.reset_index()    
    

In [58]:
ls_wt_num = list(range(193))
for i in ls_wt_num:
    if i in [176,35,82,160,196,181,12,158,182,100,43,69,145,125,96,41,64,88,154,167,174,61,111,194,99,151,171,193,156,168]:
        ls_wt_num.remove(i)

In [63]:
for i in ls_wt_num:
    if i > 100:
        try:
            drop_user(i)
        except:
            pass
      
    

101 , number of drop_user :  6756
102 , number of drop_user :  5743
103 , number of drop_user :  3894
104 , number of drop_user :  4411
105 , number of drop_user :  4002
106 , number of drop_user :  8427
107 , number of drop_user :  3846
108 , number of drop_user :  1682
109 , number of drop_user :  6988
110 , number of drop_user :  1359
112 , number of drop_user :  11284
113 , number of drop_user :  709
114 , number of drop_user :  494
115 , number of drop_user :  60007
116 , number of drop_user :  21134
117 , number of drop_user :  6878
118 , number of drop_user :  32768
119 , number of drop_user :  7586
120 , number of drop_user :  12952
121 , number of drop_user :  8107
122 , number of drop_user :  9578
123 , number of drop_user :  12919
124 , number of drop_user :  19566
126 , number of drop_user :  5752
127 , number of drop_user :  17421
128 , number of drop_user :  3918
129 , number of drop_user :  2858
130 , number of drop_user :  14121
131 , number of drop_user :  4439
132 , n

In [7]:
def set_wt(i):
    df=pd.read_csv('data/dropu_webtoon_{}.csv'.format(i)) 
    df = df.drop(['Unnamed: 0'], axis=1)
    df['wt_num'] = i
    df.to_csv('data/dropu_webtoon_{}.csv'.format(i),index=False )        
    
    return df

In [8]:
for i in range(193):
    try:
        set_wt(i)
    except:
        pass
        

In [37]:
drama = get_wt_info('drama')
daily = get_wt_info('daily')
comic = get_wt_info('comic')
fantasy = get_wt_info('fantasy')
action = get_wt_info('action')
pure = get_wt_info('pure')
sensibility = get_wt_info('sensibility')
thrill = get_wt_info('thrill')
historical = get_wt_info('historical')
sports = get_wt_info('sports')

In [96]:
pieces = [drama, daily, comic, fantasy, action, pure, sensibility, thrill, historical, sports]
info_wt = pd.concat(pieces)

In [97]:
pd.options.display.max_rows = 999
info_wt = info_wt.groupby(['title']).min()
info_wt

,score,artist,genre
title,,,
10월 28일,9.47,천정학,fantasy
1인용 기분컷툰,9.97,윤파랑,sensibility
5kg을 위하여,9.81,수오수 / 홍끼,drama
Doll 체인지,9.97,늉비,fantasy
Penguin loves ...,9.96,펭귄,daily
가담항설,9.99,랑또,comic
가우스전자 시즌3~4,9.97,곽백수,comic
간 떨어지는 동거,9.98,나,comic
간질간질,8.99,손하은,comic


In [100]:

df = pd.DataFrame(columns=['wt_num', 'title',  'score', 'artist', 'genre', 'chapter', 'reader'])
def mk_df(i):
    wt_num = i
    title = webtoons[i]['title']
    score = info_wt.loc[webtoons[i]['title']]['score']
    artist = info_wt.loc[webtoons[i]['title']]['artist']
    genre = info_wt.loc[webtoons[i]['title']]['genre']
    ff = pd.read_csv('data/dropu_webtoon_{}.csv'.format(i)).tail(1)    
    chapter = list(ff['chapter'])[0]
    reader = len(set(pd.read_csv('data/dropu_webtoon_{}.csv'.format(i))['user_id']))
    df.loc[i] = [i, title, score, artist, genre, chapter, reader]
    return df

In [103]:
for i in range(193):
    try:
        mk_df(i)
    except:
        pass

In [105]:
len(df)

139

In [110]:
df.sort_values(by = 'reader', ascending=False)

,wt_num,title,score,artist,genre,chapter,reader
86,86,연애혁명,9.87,232,comic,220,6657
115,115,외모지상주의,9.60,박태준,drama,179,3225
144,144,호랑이형님,9.98,이상규,action,300,3202
9,9,심연의 하늘 시즌5,9.92,윤인완 / 김선희,thrill,33,2835
170,170,조의 영역,9.94,조석,thrill,14,2411
89,89,기기괴괴,9.92,오성대,thrill,14,2392
1,1,뷰티풀 군바리,9.82,설이 / 윤성원,drama,161,2332
90,90,최강전설 강해효,9.71,최병열,action,243,2004
72,72,성공한 덕후,9.73,옛사람,comic,21,1822
127,127,테러맨,9.94,한동우 / 고진호,action,101,1799


In [111]:
df.to_csv('info_wt.csv',index=False)

In [12]:
df = pd.read_csv('info_wt.csv')
wt_nums = list(df['wt_num'])
df = pd.DataFrame(columns = ['chapter','user_id','comment','like','dislike','wt_num'])
for i in wt_nums:
    add = pd.read_csv('data/dropu_webtoon_{}.csv'.format(i))
    df = pd.concat([df, add])

In [14]:
comments = df.reset_index().drop(['index'],axis=1)
comments.to_csv('comments.csv',index=False)

In [15]:
comments

,chapter,user_id,comment,like,dislike,wt_num
0,365,노이****(0710****),에드안 멋있다 진짜... 눈동자 갸멋있음..,2,0,0
1,365,지그****(tkda****),갑자기 쿤을 형님이라 부르고 싶네,0,1,0
2,365,Mi****(dlfd****),라크가 혹시 고대종인가?!,3,4,0
3,365,파르****(hans****),저 그림 2부258화에 나온 라크 모습이랑 거의 일치하는데? 라크의 정체가 뭘지 진...,2,0,0
4,365,노이****(0710****),독자: 라크는 역시 대단한 악어였어!!\n포비더: 그래도 악어는 악어지.,2,0,0
5,365,신지****(zisu****),라크무엇..,0,0,0
6,365,hi****(sjhk****),항상 재밋네요 ㅎㅎ,0,0,0
7,365,차예****(nana****),쿤 가문 완전 콩가루집안인건가...,2,0,0
8,365,Ja****(gusf****),2769,0,0,0
9,365,히뭉****(haye****),라헬은 쿤을 싫어할텐데 왜 쿤을 치료한건지 이해가 안되는 부분..,2,1,0
